In [ ]:
#import packages
import numpy as np
import pandas as pd
#import ggplot as ggp

In [ ]:
#import data from csv
dataTSF = pd.read_excel("\path\to\folder",sheet_name="Basistabel")
dataSeg = pd.read_excel("\path\to\folder")

In [ ]:
#clean imported data
selectTSF = [0,2,7,8,11]
dataTSFShort = dataTSF.iloc[:,selectTSF]
columsTSF = ['product','grid','issueYear','issueMonth','income']
dataTSFShort.columns = columsTSF
selectSeg = [0,1,2,3,5,7,8]
dataSegShort = dataSeg.iloc[:,selectSeg]
columsSeg = ['segment','clientName','grid','pam','pamLocation','sector','region']
dataSegShort.columns = columsSeg

print('-------------Sizes------------')
print('size TSF:')
print(dataTSFShort.shape)
print('size Seg:')
print(dataSegShort.shape)
print('-----------HeadData-----------')
print(dataTSFShort.head(5))
print(dataSegShort.head(5))

In [ ]:
#create descriptive dataframes of segmentation and of TSF data
summaryTSF = dataTSFShort.groupby(['product','issueYear'])['income'].sum()
summaryTSF = summaryTSF.to_frame(name=None)
summarySeg = dataSegShort.groupby(['segment','sector','region'])['grid'].count()
summarySeg = summarySeg.to_frame(name=None)

print('-------------Sizes------------')
print('size summary TSF:')
print(summaryTSF.shape)
print('size summary Seg:')
print(summarySeg.shape)
print('-----------HeadData-----------')
print(summaryTSF.head(5))
print(summarySeg.head(5))

In [ ]:
#ggp.ggplot(summaryTSF, ggp.aes(x='issueYear', y='income', , color='product')) +\
#    ggp.geom_bar()

In [ ]:
#inspect final data source
fullData = pd.merge(dataTSFShort, dataSegShort, how="inner", on="grid")

#Segmented Coverage
segEUP = dataSegShort['grid'].unique()
tsfEUP = dataTSFShort['grid'].unique()
mergeEUP = fullData['grid'].unique()
print('----Coverage---')
print(float(mergeEUP.shape[0])/float(segEUP.shape[0]))
print(segEUP.shape[0])
print(tsfEUP.shape[0])
print(mergeEUP.shape[0])

companiesTSF   = dataTSFShort.grid.unique()
companiesFinal = fullData['clientName'].unique()
gridsFinal = fullData['grid'].unique()
productsFinal = fullData['product'].unique()

print('-------------Type-------------')
print('type full data:')
print(type(fullData))
print(fullData.columns)
print('-------------Size-------------')
print('size full data:')
print(fullData.size)
print('number unique companies (pre/post filter):')
print(companiesTSF.size)
print('-----------HeadData-----------')
print(fullData.head(5))
print('--------Products-----------')
print(productsFinal)


In [ ]:
#Create predictive score
compProdFinal = fullData.groupby(['grid','product']).size().reset_index().rename(columns={0:'count'})
periods = list(range(1,13))
cycles = list([2015,2016,2017])

#one company and product example
#queryString = '{0} {1}{2}{3} {4} {5}{6}{7}'.format("clientName ==","'",clients[countCompany],"'","& product ==","'",products[countProduct],"'")
#oneDFCompany = fullData.query(queryString)

s = (3,12)
frame = np.ones(s)
z = (compProdFinal.shape[0],11)
predictScore = pd.DataFrame(np.zeros(z))

#itter all products and clients
#compProdFinal.shape[0]
for clientNum in range(compProdFinal.shape[0]) :
    CompanyDF = fullData[np.logical_and(fullData['grid'] == compProdFinal.iloc[clientNum,0],fullData['product'] == compProdFinal.iloc[clientNum,1])]
    for year in range(0,3) :
        for month in range(0,12) :
            identifyIncome = CompanyDF[np.logical_and(CompanyDF['issueYear'] == cycles[year], CompanyDF['issueMonth'] == periods[month])]
            if not identifyIncome.empty :
                IncomeSwitch = 10
                frame[year,month] =  IncomeSwitch
            else :
                IncomeSwitch = 1
                frame[year,month] =  IncomeSwitch                              
    predictScore.iloc[clientNum,0] = CompanyDF.iloc[0,6]
    predictScore.iloc[clientNum,1] = CompanyDF.iloc[0,5]
    predictScore.iloc[clientNum,2] = CompanyDF.iloc[0,1]
    predictScore.iloc[clientNum,3] = CompanyDF.iloc[0,7]    
    predictScore.iloc[clientNum,4] = CompanyDF.iloc[0,8]
    predictScore.iloc[clientNum,5] = CompanyDF.iloc[0,9]
    predictScore.iloc[clientNum,6] = CompanyDF.iloc[0,10]
    predictScore.iloc[clientNum,7] = CompanyDF.iloc[0,0]
    predictScore.iloc[clientNum,8] = np.sum(frame[0,:]*frame[1,:]*frame[2,:])
    
    income = CompanyDF.iloc[:,4]
    income = income.replace(0, np.NaN)
    income = income.mean()
    predictScore.iloc[clientNum,9] = income
    predictScore.iloc[clientNum,10] = predictScore.iloc[clientNum,9]*predictScore.iloc[clientNum,8]

#Creating strings = forString = '{0} {1}{2}{3} {4} {5}{6}{7}'.format("issueYear ==","'",cycles[year],"'","& issueMonth ==","'",periods[month],"'")
# identifyIncome = CompanyDF.query(forString)['clientName'].count()  

In [ ]:
#Print result
predictScore = predictScore[(predictScore.T != 0).any()]
predictScore.columns = ['clientName','segment','grid','pam','pamLocation','sector','region','product','score','avgIncome','piScore']
print(predictScore.sort_values(by=['score','avgIncome'], ascending=False).head(5))

In [ ]:
#Export result
predictScore = predictScore.sort_values(by=['score','avgIncome'], ascending=False)
predictScore.to_excel('predictScore.xlsx')

In [ ]:
#summarize results
predictScoreResults_segRegion = predictScore.groupby(by=['segment','product'], as_index=False).agg({'score': pd.Series.mean, 'avgIncome': pd.Series.mean})
predictScoreResults_productRegion = predictScore.groupby(by=['product','region'], as_index=False).agg({'score': pd.Series.mean, 'avgIncome': pd.Series.mean})
predictScoreResults_sectorProduct = predictScore.groupby(by=['product','sector'], as_index=False).agg({'score': pd.Series.mean, 'avgIncome': pd.Series.mean, 'piScore': pd.Series.mean})
predictScoreResults_all = predictScore.groupby(by=['segment','product','region','sector'], as_index=False).agg({'score': pd.Series.mean, 'avgIncome': pd.Series.mean})

In [ ]:
print(predictScoreResults_sectorProduct.head(5))
predictScoreResults_segRegion.to_excel('predictScoreResults_segRegion.xlsx')
predictScoreResults_productRegion.to_excel('predictScoreResults_productRegion.xlsx')
predictScoreResults_sectorProduct.to_excel('predictScoreResults_sectorProduct.xlsx')
predictScoreResults_all.to_excel('predictScoreResults_all.xlsx')